In [6]:
# Install required libraries
!pip install -q transformers datasets scikit-learn torch

import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Synthetic dataset
data = {
    "text": [
        "Failed to deploy application due to missing dependency in Dockerfile",
        "Jenkins pipeline failed with timeout error",
        "Kubernetes pod is in CrashLoopBackOff state",
        "Terraform apply failed: resource already exists",
        "GitHub Actions workflow timed out waiting for runner",
        "Ansible playbook failed to connect to host",
    ],
    "label": ["Deployment", "CI/CD", "Infrastructure", "Infrastructure", "CI/CD", "Configuration"]
}

# Convert to DataFrame and map labels to integers
df = pd.DataFrame(data)
label2id = {label: i for i, label in enumerate(df["label"].unique())}
df["label"] = df["label"].map(label2id)

# Split into train/test
train_df, test_df = train_test_split(df, test_size=0.2)
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
print("Unique labels:", label2id)

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize text
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Load BERT for sequence classification
num_labels = len(label2id)
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=num_labels
).to(device)

# Define metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted"),
    }

# Training arguments (updated parameter names)
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # ✅ Changed from 'evaluation_strategy' to 'eval_strategy'
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()

# Evaluate
results = trainer.evaluate()
print("Evaluation results:", results)

# Save model
model.save_pretrained("./devops_bert_model")
tokenizer.save_pretrained("./devops_bert_model")

# Inference example
def predict(text):
    inputs = tokenizer(text, return_tensors="pt").to(device)
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)
    pred_label_id = probs.argmax().item()
    return list(label2id.keys())[list(label2id.values()).index(pred_label_id)]

# Test prediction
predict("Terraform plan failed due to state lock issue")

Using device: cuda
Unique labels: {'Deployment': 0, 'CI/CD': 1, 'Infrastructure': 2, 'Configuration': 3}


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,1.421439,0.500000,0.333333
2,No log,1.466757,0.000000,0.000000
3,No log,1.495884,0.000000,0.000000


Evaluation results: {'eval_loss': 1.4958841800689697, 'eval_accuracy': 0.0, 'eval_f1': 0.0, 'eval_runtime': 0.0994, 'eval_samples_per_second': 20.124, 'eval_steps_per_second': 10.062, 'epoch': 3.0}


'Configuration'